## Load retail csv table

In [0]:
# File location and type
file_location = "/FileStore/tables/online_retail_II.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df.limit(10))

Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01T07:45:00.000+0000,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,22041,"""RECORD FRAME 7"""" SINGLE SIZE """,48,2009-12-01T07:45:00.000+0000,2.1,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01T07:45:00.000+0000,1.65,13085.0,United Kingdom
489434,21871,SAVE THE PLANET MUG,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01T07:45:00.000+0000,5.95,13085.0,United Kingdom
489435,22350,CAT BOWL,12,2009-12-01T07:46:00.000+0000,2.55,13085.0,United Kingdom
489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01T07:46:00.000+0000,3.75,13085.0,United Kingdom


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

In [0]:
# Create a view or table

rdf = "online_retail_II_csv"

df.createOrReplaceTempView(rdf)

In [0]:
from pyspark.sql.functions import col, avg, min, max

In [0]:
from pyspark.sql import functions as F

In [0]:
list(df.columns)

Out[4]: ['Invoice',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'Price',
 'Customer ID',
 'Country']

In [0]:
nInvoice = df.Invoice
stockcode = df.StockCode
description = df.Description
qty = df.Quantity
date = df.InvoiceDate
price = df.Price
#customer_id = df."Customer_ID"
country = df.Country

## Total Invoice Amount Distribution

<h5> Distribution (Invoice Amount) </h5>

In [0]:
amount_df = df.withColumn('amount', (qty * price))
amount_sum = amount_df.groupby(nInvoice).sum('amount').where('sum(amount) > 0')
amount_min = amount_sum.agg({'sum(amount)' : 'min'})
amount_max = amount_sum.agg({'sum(amount)' : 'max'})
amount_mean = amount_sum.agg({'sum(amount)' : 'mean'})
amount_median = amount_sum.approxQuantile('sum(amount)', [0.5], 0)[0]
amount_mode = amount_sum.groupby('sum(amount)').count().orderBy("count", ascending=False).select('sum(amount)').first()[0]

amount_distribution_invoice = amount_sum.select(F.max('sum(amount)').alias("Maximum"),\
                                               F.min('sum(amount)').alias("Minimum"),\
                                               F.mean('sum(amount)').alias("Mean"))\
.withColumn("Median", F.lit(amount_median))\
.withColumn("Mode", F.lit(amount_mode))

amount_distribution_invoice.show()

+--------+-------+-----------------+-----------------+----+
 Maximum|Minimum| Mean| Median|Mode|
+--------+-------+-----------------+-----------------+----+
168469.6| 0.19|523.3037611158244|304.3100000000002|15.0|
+--------+-------+-----------------+-----------------+----+

<h5> Distribution (Remove Outliers) </h5>

In [0]:
outlier_quantile = amount_sum.approxQuantile('sum(amount)', [0.85], 0)[0]
outlier_amount = amount_sum.where(F.col('sum(amount)') < outlier_quantile)
outlier_mode = outlier_amount.groupby('sum(amount)').count().orderBy("count", ascending=False).select('sum(amount)').first()[0]
outlier_median = outlier_amount.approxQuantile('sum(amount)', [0.5], 0)[0]

distribution_outlier_data = outlier_amount.select(F.max('sum(amount)').alias("Maximum"),\
                                               F.min('sum(amount)').alias("Minimum"),\
                                               F.mean('sum(amount)').alias("Mean"))\
.withColumn("Median", F.lit(outlier_median))\
.withColumn("Mode", F.lit(outlier_mode))

distribution_outlier_data.show()

+-----------------+-------+-----------------+------+----+
 Maximum|Minimum| Mean|Median|Mode|
+-----------------+-------+-----------------+------+----+
724.2499999999999| 0.19|271.6838912992427|256.22|15.0|
+-----------------+-------+-----------------+------+----+

## Monthly Placed and Canceled Orders

In [0]:
orders_df = amount_df.withColumn("Date", F.date_format(date, "YYYYMM"))
orders_total = orders_df.groupby(nInvoice, "Date").count().select(nInvoice, "Date", "count")
orders_placed = orders_total.groupby("Date").count().select("Date", "count").orderBy("Date", ascending=True)
orders_canceled = orders_total.where(nInvoice.contains("C")).groupby("Date").count().select("Date", "count").orderBy("Date", ascending=True)
orders_placed_canceled = orders_placed.select("Date", F.col("count").alias("Placement")).join(orders_canceled, "Date")

orders_displacement = orders_placed_canceled.select("Date", "Placement", F.col("count").alias("Cancellation"))\
.orderBy("Date", ascending=True)

orders_displacement.show()

+------+---------+------------+
 Date|Placement|Cancellation|
+------+---------+------------+
200912| 2330| 401|
201001| 1633| 300|
201002| 1969| 240|
201003| 2367| 407|
201004| 1892| 304|
201005| 2418| 407|
201006| 2216| 357|
201007| 2017| 344|
201008| 1877| 273|
201009| 2375| 371|
201010| 2965| 476|
201011| 3669| 576|
201012| 2025| 326|
201101| 1476| 260|
201102| 1393| 219|
201103| 1983| 318|
201104| 1744| 240|
201105| 2162| 314|
201106| 2012| 329|
201107| 1927| 270|
+------+---------+------------+
only showing top 20 rows

## Monthly Sales

In [0]:
sales_df = orders_df.where('amount > 0').groupby("Date").sum('amount').orderBy("Date", ascending=True)

sales_monthly = sales_df.select("Date", F.col('sum(amount)').alias("Monthly Sales"))\
.withColumn("Sales(Millions)", F.col("Monthly Sales")/1000000)\
.drop("Monthly Sales")

sales_monthly.show()

+------+------------------+
 Date| Sales(Millions)|
+------+------------------+
200912|0.8256857600000115|
201001|0.6527085019999943|
201002|0.5537133060000008|
201003| 0.833570130999969|
201004|0.6815289919999798|
201005|0.6598588599999907|
201006|0.7522701399999794|
201007|0.6507129400000107|
201008|0.6972749099999849|
201009|0.9243330109999602|
201010| 1.165483909999892|
201011|1.4702724819998032|
201012|1.2625987899999327|
201101|0.6913645600000108|
201102|0.5236318900000277|
201103|0.7176393600000187|
201104|0.5378086210000242|
201105|0.7705360200000108|
201106|0.7617399000000218|
201107|0.7192211910000272|
+------+------------------+
only showing top 20 rows

## Monthly Sales Growth

In [0]:
from pyspark.sql.window import Window

growth_df = sales_df.withColumn("Growth %", F.col("sum(amount)")/F.sum("sum(amount)").over(Window.partitionBy()))\
.orderBy("Date", ascending=True)\
.drop("sum(amount)")

growth_df.show()

+------+--------------------+
 Date| Growth %|
+------+--------------------+
200912| 0.03936904660165929|
201001| 0.03112141770803537|
201002|0.026401284851845522|
201003| 0.03974497674888878|
201004| 0.03249559087276582|
201005| 0.03146234980467272|
201006|0.035868558758595445|
201007|0.031026268467028894|
201008| 0.03324636290924626|
201009| 0.04407258929293994|
201010| 0.05557076625164177|
201011| 0.07010321440082327|
201012| 0.06020124484489752|
201101| 0.03296455491902357|
201102|0.024966990201606164|
201103|0.034217348506803955|
201104|0.025642942737590362|
201105| 0.03673948365009598|
201106| 0.03632008092454413|
201107|0.034292770878573194|
+------+--------------------+
only showing top 20 rows

## Monthly Active Users

In [0]:
monthlyUsers_df = orders_df.select("Date", "Customer Id").distinct().groupBy("Date").count().orderBy("Date", ascending=True)
users_monthly = monthlyUsers_df.select("Date", F.col("count").alias("# of Active Users"))

users_monthly.show()

+------+-----------------+
 Date|# of Active Users|
+------+-----------------+
200912| 1046|
201001| 787|
201002| 808|
201003| 1112|
201004| 999|
201005| 1063|
201006| 1096|
201007| 989|
201008| 965|
201009| 1203|
201010| 1578|
201011| 1684|
201012| 949|
201101| 784|
201102| 799|
201103| 1021|
201104| 900|
201105| 1080|
201106| 1052|
201107| 994|
+------+-----------------+
only showing top 20 rows

## New and Existing Users

In [0]:
user_earliest = orders_df.groupby("Customer Id").agg(F.min("Date").alias("SignupDate"))\
.groupby("SignupDate").count().sort("SignupDate").withColumnRenamed("count", "New Users")

user_list = users_monthly.join(user_earliest, user_earliest.SignupDate==users_monthly.Date)\
.withColumn("Existing Users", F.col("# of Active Users")-F.col("New Users"))\
.orderBy("Date", ascending=True)\
.drop("SignupDate")


user_list.show()

+------+-----------------+---------+--------------+
 Date|# of Active Users|New Users|Existing Users|
+------+-----------------+---------+--------------+
200912| 1046| 1046| 0|
201001| 787| 394| 393|
201002| 808| 363| 445|
201003| 1112| 436| 676|
201004| 999| 291| 708|
201005| 1063| 254| 809|
201006| 1096| 269| 827|
201007| 989| 183| 806|
201008| 965| 158| 807|
201009| 1203| 242| 961|
201010| 1578| 379| 1199|
201011| 1684| 322| 1362|
201012| 949| 77| 872|
201101| 784| 71| 713|
201102| 799| 123| 676|
201103| 1021| 178| 843|
201104| 900| 105| 795|
201105| 1080| 108| 972|
201106| 1052| 108| 944|
201107| 994| 102| 892|
+------+-----------------+---------+--------------+
only showing top 20 rows

## Finding RFM

In [0]:
from datetime import datetime
today = datetime(2022,1,1)

orders_df = orders_df.withColumn('TotalPrice', F.round(orders_df.Quantity * orders_df.Price, 2 ))
orders_df = orders_df.withColumn('Duration', F.datediff(F.lit(today), 'InvoiceDate'))

recency = orders_df.groupBy('Customer ID').agg(F.min('Duration').alias('Recency'))
frequency = orders_df.groupBy('Customer ID', 'Invoice').count().groupBy('Customer ID').agg(F.count("*").alias("Frequency"))
monetary = orders_df.groupBy('Customer ID').agg(F.round(F.sum('TotalPrice'), 2).alias('Monetary'))

rfm = recency.join(frequency,'Customer ID', how = 'inner').join(monetary,'Customer ID', how = 'inner')

rfm.show()

+-----------+-------+---------+--------+
Customer ID|Recency|Frequency|Monetary|
+-----------+-------+---------+--------+
 12467.0| 4062| 2| 0.0|
 12493.0| 3841| 3| 416.79|
 12671.0| 4282| 1| 2622.48|
 12737.0| 4174| 2| 3710.5|
 13094.0| 3697| 17| 2214.66|
 13533.0| 3858| 3| 270.79|
 13607.0| 3716| 3| 1060.61|
 13918.0| 3725| 2| 1212.84|
 13956.0| 3681| 5| 1026.42|
 13973.0| 3963| 1| 264.7|
 14024.0| 3797| 3| 645.74|
 14094.0| 4115| 2| 334.27|
 14269.0| 4111| 2| 261.68|
 14285.0| 3697| 9| 3158.64|
 14452.0| 3686| 4| 665.59|
 14473.0| 3750| 2| 234.34|
 14768.0| 3693| 3| 139.5|
 15776.0| 3809| 1| 241.62|
 15846.0| 4061| 1| 107.01|
 15893.0| 4157| 1| 305.28|
+-----------+-------+---------+--------+
only showing top 20 rows

## RFM Segmentation

In [0]:
def RScore(x):
    if  x <= 16:
        return 1
    elif x<= 50:
        return 2
    elif x<= 143:
        return 3
    else:
        return 4

def FScore(x):
    if  x <= 1:
        return 4
    elif x <= 3:
        return 3
    elif x <= 5:
        return 2
    else:
        return 1

def MScore(x):
    if  x <= 293:
        return 4
    elif x <= 648:
        return 3
    elif x <= 1611:
        return 2
    else:
        return 1

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType

R_udf = F.udf(lambda x: RScore(x), StringType())
F_udf = F.udf(lambda x: FScore(x), StringType())
M_udf = F.udf(lambda x: MScore(x), StringType())

rfm_seg = rfm.withColumn("r_seg", R_udf("Recency"))
rfm_seg = rfm_seg.withColumn("f_seg", F_udf("Frequency"))
rfm_seg = rfm_seg.withColumn("m_seg", M_udf("Monetary"))


rfm_seg = rfm_seg.withColumn('RFMScore', F.concat(F.col('r_seg'),F.col('f_seg'), F.col('m_seg'))).sort(F.col('RFMScore'))

rfm_seg.show()

+-----------+-------+---------+---------+-----+-----+-----+--------+
Customer ID|Recency|Frequency| Monetary|r_seg|f_seg|m_seg|RFMScore|
+-----------+-------+---------+---------+-----+-----+-----+--------+
 17884.0| 3679| 23| 3028.89| 4| 1| 1| 411|
 17063.0| 3684| 23| 3576.88| 4| 1| 1| 411|
 17633.0| 3707| 9| 1974.89| 4| 1| 1| 411|
 16208.0| 3720| 10| 3440.45| 4| 1| 1| 411|
 13458.0| 3683| 24| 11850.62| 4| 1| 1| 411|
 14064.0| 3705| 21| 3039.75| 4| 1| 1| 411|
 12830.0| 3713| 7| 6748.4| 4| 1| 1| 411|
 13763.0| 3739| 6| 5181.99| 4| 1| 1| 411|
 18210.0| 3677| 7| 2768.83| 4| 1| 1| 411|
 14306.0| 3727| 7| 3141.5| 4| 1| 1| 411|
 16600.0| 3680| 16| 2093.56| 4| 1| 1| 411|
 17659.0| 3679| 17| 3784.13| 4| 1| 1| 411|
 15311.0| 3676| 270|113513.07| 4| 1| 1| 411|
 15372.0| 3813| 10| 5954.42| 4| 1| 1| 411|
 12609.0| 3754| 10| 3417.86| 4| 1| 1| 411|
 15750.0| 3678| 24| 6692.55| 4| 1| 1| 411|
 14211.0| 3729| 26| 4301.69| 4| 1| 1| 411|
 15898.0| 3677| 29| 5534.36| 4| 1| 1| 411|
 14233.0| 3719| 10| 2766.96| 4| 1| 1| 411|
 16353.0| 3679| 41| 9985.93| 4| 1| 1| 411|
+-----------+-------+---------+---------+-----+-----+-----+--------+
only showing top 20 rows